In [7]:
import csv
import json
import os
import math
import openpyxl
import re
import xlsxwriter

import pandas as pd
import numpy as np
from collections import Counter
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

from sklearn.preprocessing import LabelEncoder


In [8]:
def readCSVFile(filePath):
        with open(filePath, 'r', newline='',  encoding='utf-8') as csvfile:
            has_header = csv.Sniffer().has_header(csvfile.readline())
            csvfile.seek(0)  # Rewind.
            dialect = csv.Sniffer().sniff(csvfile.read(), delimiters=';,\t')
            csvfile.seek(0) 
            reader = csv.reader(csvfile, dialect)
            if(has_header):
                next(reader)  # Skip header row.
            dataset = pd.DataFrame(reader)
        return dataset
        #print(filePath)


In [9]:
def readExcel(filePath):
    dataset = pd.read_excel(filePath)
    return dataset

In [10]:
def custom_csv(fname):
    if fname.endswith((".data", ".csv")):
        return readCSVFile(fname)
    elif fname.endswith((".xlsx")):
        return readExcel(fname)

In [11]:
def getLabels(dataset):
    try:
        flag = 0
        #dataset = custom_csv(filePath)
        n = dataset.iloc[:, -1].nunique(dropna=False)
        perc = dataset.iloc[:, -1].value_counts(normalize=True)*100
        if(len(perc) > len(dataset.iloc[:, 0].value_counts(normalize=True)*100)):  #checking whether 1st column is label
            n=dataset.iloc[:, 0].nunique(dropna=False)
            flag = 1
        if(flag == 1):
            return dataset.iloc[:, 0]
        else:
            return dataset.iloc[:,-1]
    except:
        print("Can not read last column items for", filePath)

In [12]:
def countUniqueLabels(dataset):
    try:
       # dataset = custom_csv(filePath)
        n = getLabels(dataset)
        #perc = dataset.iloc[:, -1].value_counts(normalize=True)*100
        #if(len(perc) > len(dataset.iloc[:, 0].value_counts(normalize=True)*100)):  #checking whether 1st column is label
           # n=dataset.iloc[:, 0].nunique(dropna=False)
        return n.nunique(dropna=False)
    except:
        print("Can not read unique items for", filePath)

In [13]:
def computeClassEntropy(dataset):
   # dataset = custom_csv(filePath)
    classLabel = getLabels(dataset)
    entropy=0
    rows = readRows(dataset)
    uc = countUniqueLabels(dataset)
    values, counts = np.unique(classLabel, return_counts=True)
    for i in range(len(values)):
        p = counts[i] / rows
        entropy -= p * math.log(p,uc)
    return entropy

In [14]:
def readRows(dataset):
    try:
       # dataset = custom_csv(filePath)
        return len(dataset.axes[0])
    except:
        print("Can not read rows for",filePath)

In [15]:
def readColumns(dataset):
    try:
       # dataset = custom_csv(filePath)
        return len(dataset.axes[1])
    except:
        print("Can not read columns for",filePath)

In [16]:
def computeCorrelation(dataset):
    sp=p=sn=n=0
       #dataset = custom_csv(filePath)
    rows, cols = dataset.shape
    corr1 = dataset.corr() #Compute pairwise correlation of columns, excluding NA/null values.

    c1 = corr1.unstack()
    print("Cor",corr1)
    for i in c1:
        if(i==1):
            sp+=1
        elif(i==-1):
            sn+=1
        elif(i>0):
            p+=1
        elif(i<=0):
            n+=1
    corrDict = {}
    sp=sp/(cols*(cols-1))
    corrDict['spCorr'] = sp 
    p=p/(cols*(cols-1))
    corrDict['pCorr'] = p
    sn=sn/(cols*(cols-1))
    corrDict['snCorr'] = sn 
    n=n/(cols*(cols-1))
    corrDict['nCorr'] = n
        
    return corrDict


In [17]:
def computeCorrelation(dataset):
    sp=p=sn=n=0
       #dataset = custom_csv(filePath)
    rows, cols = dataset.shape
    corr1 = dataset.corr() #Compute pairwise correlation of columns, excluding NA/null values.

    c1 = corr1.unstack()
    for i in c1:
        if(i==1):
            sp+=1
        elif(i==-1):
            sn+=1
        elif(i>0):
            p+=1
        elif(i<=0):
            n+=1
    corrDict = {}
    sp=sp/(cols*(cols-1))
    corrDict['spCorr'] = sp 
    p=p/(cols*(cols-1))
    corrDict['pCorr'] = p
    sn=sn/(cols*(cols-1))
    corrDict['snCorr'] = sn 
    n=n/(cols*(cols-1))
    corrDict['nCorr'] = n
        
    return sp,p,sn,n


In [18]:
def computeClassOverlap(dataset):
    m = 0
    s = 0 
    count = 0
    outlier = 0
    flag = 0
   # dataset = custom_csv(filePath)
    km = KMeans(n_clusters = countUniqueLabels(dataset))
    clusters = km.fit_predict(dataset)
    # points array will be used to reach the index easy
    points = np.empty((0,len(dataset.axes[1])), float)
    # distances will be used to calculseetate outliers
    distances = np.empty((0,len(dataset.axes[0])), float)   
        # getting points and distances
    centroids = km.cluster_centers_
    for i, center_elem in enumerate(centroids):
            # cdist is used to calculate the distance between center and other points
        distances = np.append(distances, cdist([center_elem],dataset[clusters == i], 'euclidean')) 
        points = np.append(points, dataset[clusters == i], axis=0)
        
    cluster_distance_d = {'cluster':clusters, 'distance':distances}
    cluster_distance = pd.DataFrame(cluster_distance_d)

    grouped = cluster_distance.groupby(['cluster'], as_index = False)
    cluster_statistics = grouped[['distance']].agg([np.mean, np.std]) 
    
    for i in range(len(cluster_distance)):#
        for j in range(len(cluster_statistics)):
            if(cluster_statistics.index[j]==cluster_distance.iloc[i,0]):
                m = cluster_statistics.iloc[j,0]
                s =cluster_statistics.iloc[j,1]
                flag=1
                break
            if(flag==1):
                if(cluster_distance.iloc[i,1] > (m + 3 * s)):
                    outlier+=1
                    for k in range(len(cluster_statistics)):
                        if(cluster_statistics.index[k]!=cluster_distance.iloc[i,0]):
                            dist = cdist([points[i]], [centroids[k]], 'euclidean')
                            m1 = cluster_statistics.iloc[k,0]
                            s1 = cluster_statistics.iloc[k,1]
                            if(dist <= (m1 + 3 * s1)):
                                count+=1
        
    #print(count)
    #print(outlier)
    return [count/(dataset.shape[0] * dataset.shape[1]), outlier/(dataset.shape[0] * dataset.shape[1])]

In [19]:
def completeness(dataset):
    #dataset = custom_csv(filePath)
    #dataset1 = dataset.dropna(how = 'all', inplace=False)
    totalMissing = dataset.isnull().sum().sum()
    return (totalMissing /(len(dataset.axes[1]) * len(dataset.axes[0])))

In [20]:
def classimbalanceRatio(dataset):
    #dataset = custom_csv(filePath)
    totalClasses = countUniqueLabels(dataset)
    perc = dataset.iloc[:, -1].value_counts(normalize=True)*100
    if(len(perc) > len(dataset.iloc[:, 0].value_counts(normalize=True)*100)):
        perc=dataset.iloc[:, 0].value_counts(normalize=True)*100
    count = 0
    for idx, item in enumerate(perc):
        for j in perc[idx+1:]:
            if(abs(item-j) > 30):
                count+=abs(item-j)
    #print("count",(count))
    return (count/(dataset.shape[0]*dataset.shape[1]))

In [21]:
def conciseness(dataset):
    #dataset = custom_csv(filePath)
    uniques = dataset.drop_duplicates(keep='first')
    return (1 - (uniques.shape[0] * uniques.shape[1]) /(dataset.shape[0] * dataset.shape[1]))

In [22]:
def typeCheck(singleCol):
    ci=cs=co=cf=cd=cu=0
    intType = re.compile(r"^\d+$")
    dateType1 = re.compile(r"[0-9]{4}[-/][0-9]?[0-9]?[-/][0-9]?[0-9]?")
    dateType2 = re.compile(r"[0-9]?[0-9]?[-/][0-9]?[0-9]?[-/][0-9]{4}")
    stringType = re.compile("^[a-zA-Z]+.*\s*[a-zA-Z]*$")
    floatType = re.compile(r"[-+]?[0-9]*\.?[0-9]*")
    uriType = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")

    for i in range(len(singleCol)):
        if((uriType.match(str(singleCol[i])))):
            cu+=1
        elif(stringType.match(str(singleCol[i]))):
            cs+=1
        elif((intType.match(str(singleCol[i])))):
            ci+=1
        elif(dateType1.match(str(singleCol[i]) or dateType2.match(str(singleCol[i])))):
            cd+=1
        elif(floatType.match(str(singleCol[i]))):
            cf+=1
        else:
            co+=1
    daConsidered=['int','str','float','date','uri','other']
    #overall=[ci,cs,cf,cd,cu,co]
    if(cf > ci):             #column with float values, int gets assigned to ci, coverting it to cf
        cf = cf+ci
        ci=0
    #return overall.index(max(overall))
    overall=[ci,cs,cf,cd,cu,co]

    return max(overall)

In [23]:
def conditional_entropy(x, y):
    "Returns H(X|Y)."
    uy, uyc = np.unique(y, return_counts=True)
    prob_uyc = uyc/float(sum(uyc))
    cond_entropy_x = np.array([entropy(x[y == v]) for v in uy])
    return prob_uyc.dot(cond_entropy_x)

In [24]:
def mutual_information(x, y):
    " Returns the information gain/mutual information [H(X)-H(X|Y)] between two random vars x & y."
    return entropy(x) - conditional_entropy(x, y)

In [25]:
def snr(dataset):    
 #   dataset = custom_csv(filePath)
    classLabel = getLabels(dataset)
    columns = list(dataset)
    mi=0
    for i in range(len(columns)):
        mi+=conditional_entropy(classLabel,dataset.iloc[:,i]) 
    e=0
    for i in columns:
        e+=entropy(dataset.iloc[:,i])
    return (e/dataset.shape[1] - mi/dataset.shape[1])/(mi/dataset.shape[1])
        

In [26]:
def entropy(vec, base=2):
    q, vec = np.unique(vec, return_counts=True)
    prob_vec = np.array(vec/float(sum(vec)))
    if base == 2:
        logfn = np.log2
    elif base == 10:
        logfn = np.log10
    else:
        logfn = np.log
    return prob_vec.dot(-logfn(prob_vec))

In [27]:
def enattributes(dataset):
    cEntropy = computeClassEntropy(dataset)
  #  dataset = custom_csv(filePath)
    columns = list(dataset)
    mi=0
    for i in range(len(columns)-1):
        col = list(dataset.iloc[:,(i+1):])
        for c in col:
            mi+=conditional_entropy(dataset.iloc[:,i],dataset.iloc[:,c])  
    return (cEntropy / (mi / dataset.shape[1]))

In [28]:
def syntaxAccuracy(dataset):
    #dataset = custom_csv(filePath)
    count = 0
    invalid = 0
    for i in range((dataset.shape[1])):
        flag=0
        if(dataset.iloc[:, i].dtype == "object"):
            count = typeCheck(dataset.iloc[:, i])
            if(count != dataset.shape[0]):
                invalid+=1
    return (invalid/dataset.shape[1])

In [29]:
def convertstrtointcategory(df): 
    le = LabelEncoder()
    ass1 = _assumption1categorical(df) 
    ass2 = _assumption2categorical(df)

    #extract only columns that belong to 
    commonidx = (list(set(ass1) | set(ass2)))

    for i in commonidx:
        df.iloc[:,i] = le.fit_transform(df.iloc[:,i])

    return df

In [30]:
def categoryIndex(df): 
    le = LabelEncoder()
    ass1 = _assumption1categorical(df) 
    ass2 = _assumption2categorical(df)

    #extract only columns that belong to 
    commonidx = (list(set(ass1) | set(ass2)))
   # for i in commonidx:
     #   df.iloc[:,i] = le.fit_transform(df.iloc[:,i])

    return commonidx

In [31]:
def _assumption1categorical(df):
    likely_cat = []
    for idx, var in enumerate(df.columns):
        if(1.*df[var].nunique()/df[var].count() < 0.05): #or some other threshold
            likely_cat.append(idx)
    return likely_cat


In [32]:
def _assumption2categorical(df):
    top_n = 10 
    likely_cat = []
    for idx, var in enumerate(df.columns):
        if(1.*df[var].value_counts(normalize=True).head(top_n).sum() > 0.8): #or some other threshold
            likely_cat.append(idx)
    return likely_cat

In [33]:
def computeAsymmetry(df):
    mean = []
    median = []
    stdDev = []
    symmetrical = moderasymmetrical = strongasymmetric = 0
    #for col in df.columns:
    #print(df.shape)
    #print(df.median())
    for col in df:
        mean.append(df[col].astype(np.float).mean())
        stdDev.append(df[col].astype(np.float).std())
        median.append(df[col].astype(np.float).median())

    
    asym = (3*(np.array(mean) - np.array(median))) / np.array(stdDev)
    for i in asym:
        if(i<0.15):
            symmetrical+=1
        elif(i>=0.15 and i<1):
            moderasymmetrical+=1
        else:
            strongasymmetric+=1
    symmetrical/=df.shape[1]
    moderasymmetrical /=df.shape[1]
    strongasymmetric /=df.shape[1]

    return [symmetrical, moderasymmetrical, strongasymmetric]


In [34]:
listofFiles={}
for path, subdirs, files in os.walk(os.getcwd()+'/datasets/working'):
    for name in files:
        if name.endswith((".data", ".csv", ".xlsx")):
            listofFiles[name]=os.path.join(path, name)

corrDict = {}
dataCharQuality = []

temp = []
count = []
categoryIdx = []
for eachFile in listofFiles:
    temp = []
    temp.append(eachFile)
    print(eachFile)
    dataset = custom_csv(listofFiles[eachFile])

    temp.append(completeness(dataset))
    temp.append(classimbalanceRatio(dataset))
    temp.append(conciseness(dataset))
    temp.append(syntaxAccuracy(dataset))

    dataset = convertstrtointcategory(dataset)

    count = computeClassOverlap(dataset)
    temp.append(count[0])
    temp.append(count[1])
    temp.append(readRows(dataset))
    temp.append(readColumns(dataset))
    temp.append(countUniqueLabels(dataset))
    temp.append(computeClassEntropy(dataset))
    temp.append(snr(dataset))
    temp.append(enattributes(dataset))
    categoryIdx = categoryIndex(dataset)
    asymetry = computeAsymmetry(dataset.drop(categoryIdx, axis=1))
    temp.extend(asymetry)

    corrDict = computeCorrelation(dataset)
   # entropyDataframe = groupByColumnEntropy(listofDataFiles[eachFile])
    if(corrDict):
      temp.extend(corrDict)
    dataCharQuality.append(temp)

Wholesale customers data.csv
caesarian.csv
bank-full.csv
data_banknote_authentication.csv
heart_failure_clinical_records_datase.csv
wine.data
HCV-Egy-Data2.csv
LasVegasTripAdvisorReviews-Dataset.csv
iris.data
glass.data


In [37]:
with xlsxwriter.Workbook('test.xlsx') as workbook:
    worksheet = workbook.add_worksheet()

    for row_num, data in enumerate(dataCharQuality):
        worksheet.write_row(row_num, 0, data)

In [45]:
# opening the csv file in 'w+' mode

#write file using xlsx - pending
file = open('qwe.csv', 'w+', newline ='')
headerInfo = ['fName','completeness','imbalanceRatio', 'conciseness', 'syntaxAccuracy','classOverlap','outlierDetection','instances','attributes','uniqueClasses', 'entropy','snr','ena','symmetrical','modereateasymmetrical','strongasymmetric',  'strongPositive', 'positive', 'strongNegative', 'negative']

# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerow(headerInfo)
    write.writerows(dataCharQuality)

In [54]:
listofFiles={}
for path, subdirs, files in os.walk(os.getcwd()+'/datasets/working'):
    for name in files:
        if name.endswith((".data", ".csv", ".xlsx")):
            listofFiles[name]=os.path.join(path, name)
       # elif name.endswith((".xls", ".xlsx")):
        #    listofExcelFiles[name]=os.path.join(path, name)

#for key in listofCSVFiles:
 #   readCSVFile(listofCSVFiles[key])

#for key in listofExcelFiles:
 #   readExcel(listofExcelFiles[key])

corrDict = {}
dataCharQuality = {}
count = []
categoryIdx = []
for eachFile in listofFiles:
    dataCharQuality['fName'] = {eachFile}
    print(eachFile)
    dataset = custom_csv(listofFiles[eachFile])


    dataCharQuality[eachFile]['completeness'] = completeness(dataset)
    dataCharQuality[eachFile]['imbalanceRatio'] = classimbalanceRatio(dataset)
    dataCharQuality[eachFile]['conciseness'] = conciseness(dataset)
    dataCharQuality[eachFile]['syntaxAccuracy'] = syntaxAccuracy(dataset)

    dataset = convertstrtointcategory(dataset)

    count = computeClassOverlap(dataset)
    dataCharQuality[eachFile]['classOverlap'] =  count[0]
    dataCharQuality[eachFile]['outlierDetection'] = count[1]
    dataCharQuality[eachFile]['instances'] = readRows(dataset)
    dataCharQuality[eachFile]['attributes'] = readColumns(dataset)
    dataCharQuality[eachFile]['uniqueClasses'] = countUniqueLabels(dataset)
    dataCharQuality[eachFile]['entropy'] = computeClassEntropy(dataset)
    dataCharQuality[eachFile]['snr'] = snr(dataset)
    dataCharQuality[eachFile]['ena'] = enattributes(dataset)
    categoryIdx = categoryIndex(dataset)
    asymetry = computeAsymmetry(dataset.drop(categoryIdx, axis=1))
    dataCharQuality[eachFile]['symmetrical'] = asymetry[0]
    dataCharQuality[eachFile]['moderasymmetrical'] = asymetry[1]
    dataCharQuality[eachFile]['strongasymmetric'] = asymetry[2]

    corrDict = computeCorrelation(dataset)
    #print(dataCharacteristics[eachFile]['entropy'])
   # entropyDataframe = groupByColumnEntropy(listofDataFiles[eachFile])
    if(corrDict):
        dataCharQuality[eachFile].update(corrDict)
    

Wholesale customers data.csv


NameError: name 'dataC' is not defined

In [114]:
with open("dataquality.json","w") as f:
    json.dump(dataCharQuality,f)

In [30]:
#Not used this
def kmeans(X,k,max_iterations=100):
    '''
    X: multidimensional data
    k: number of clusters
    max_iterations: number of repetitions before clusters are established
    
    Steps:
    1. Convert data to numpy aray
    2. Pick indices of k random point without replacement
    3. Find class (P) of each data point using euclidean distance
    4. Stop when max_iteration are reached of P matrix doesn't change
    
    Return:
    np.array: containg class of each data point
    '''
    if isinstance(X, pd.DataFrame):X = X.values
    idx = np.random.choice(len(X), k, replace=False)
    centroids = X[idx, :]
    P = np.argmin(distance.cdist(X, centroids, 'euclidean'),axis=1)
    for _ in range(max_iterations):
        centroids = np.vstack([X[P==i,:].mean(axis=0) for i in range(k)])
        tmp = np.argmin(distance.cdist(X, centroids, 'euclidean'),axis=1)
        if np.array_equal(P,tmp):break
        P = tmp
    return P